# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

You can check the output of `!nvidia-smi` to check which GPU you have.  Please uncomment the cell below if you'd like to do that.  Currently, RAPIDS runs on all available Colab GPU instances.

In [ ]:
# !nvidia-smi

#Setup:
This set up script:

1. Checks to make sure that the GPU is RAPIDS compatible
1. Installs the **current stable version** of RAPIDSAI's core libraries using pip, which are:
  1. cuDF
  1. cuML
  1. cuGraph
  1. cuSpatial
  1. cuxFilter
  1. cuCIM
  1. xgboost

**This will complete in about 5-6 minutes**

If you require installing the **nightly** releases of RAPIDSAI, please use the [RAPIDS Conda Colab Template notebook](https://colab.research.google.com/drive/1TAAi_szMfWqRfHVfjGSqnGVLr_ztzUM9) and use the nightly parameter option when running the RAPIDS installation cell.


In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py


Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 438, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 438 (delta 121), reused 57 (delta 54), pack-reused 269
Receiving objects: 100% (438/438), 118.31 KiB | 16.90 MiB/s, done.
Resolving deltas: 100% (223/223), done.
***********************************************************************
We will now install RAPIDS via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.2/955.2 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.8/142.8 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# RAPIDS is now installed on Colab.  
You can copy your code into the cells below or use the below to validate your RAPIDS installation and version.  
# Enjoy!

In [ ]:
import cudf
cudf.__version__

'23.12.01'

In [ ]:
import cuml
cuml.__version__

'23.12.00'

In [ ]:
import cugraph
cugraph.__version__

'23.12.00'

In [ ]:
import cuspatial
cuspatial.__version__

'23.12.01'

In [ ]:
import cuxfilter
cuxfilter.__version__

'23.12.00'

# Next Steps #

For an overview of how you can access and work with your own datasets in Colab, check out [this guide](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92).

For more RAPIDS examples, check out our RAPIDS notebooks repos:
1. https://github.com/rapidsai/notebooks
2. https://github.com/rapidsai/notebooks-contrib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

seed = 42
np.random.seed(seed)
random.seed(seed)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Progetto Data Analytics/train.csv')
df = df.drop_duplicates()
print(df.shape)
df.head()

(252123, 91)


,Year,S0,S1,S2,S3,S4,S5,S6,S7,S8,...,S80,S81,S82,S83,S84,S85,S86,S87,S88,S89
0,2007,44.76752,114.82099,3.83239,27.99928,1.49153,-15.90853,28.24844,3.61650,-7.24653,...,-1.89619,-471.02844,411.56205,443.01198,19.30254,309.07806,-336.91706,-14.70547,-474.44157,31.32820
1,2004,52.28942,75.73319,11.35941,-6.20582,-27.64559,-30.75995,12.50955,7.47877,9.88498,...,4.57060,1.36110,-6.52977,59.48672,3.69790,-36.92252,44.08077,3.39993,-70.07591,3.86143
2,2005,33.81773,-139.07371,134.19332,17.85216,63.47408,-25.28005,-34.65911,-5.99135,1.27848,...,54.16608,15.04530,39.09107,39.03041,3.68708,-61.88547,45.68115,6.39822,3.24471,35.74749
3,1998,41.60866,3.17811,-3.97174,23.53564,-19.68553,20.74407,18.80866,6.24474,-7.98424,...,28.08591,295.88684,54.02395,102.02880,40.47711,15.10258,-250.32293,2.81288,56.05172,3.60432
4,1987,44.49525,-32.25270,58.08217,3.73684,-32.53274,-18.72885,-15.85665,-3.34607,22.63786,...,31.44988,-136.50457,-85.11989,-74.96342,9.56921,-100.61689,-133.29315,9.19246,-97.37953,30.11015


In [30]:
gdf = cudf.DataFrame.from_pandas(df)

numerical_gdf = gdf.select_dtypes(include=[np.number]).drop('Year', axis=1)

tsne = cuml.TSNE(n_components=2, method='fft')
tsne_results = tsne.fit_transform(numerical_gdf)

tsne_df = pd.DataFrame(tsne_results.to_numpy(), columns=['tsne-2d-one', 'tsne-2d-two'])
tsne_df['Year'] = df['Year'].values


/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning:

Starting from version 22.04, the default method of TSNE is 'fft'.



[W] [18:44:21.711624] # of Nearest Neighbors should be at least 3 * perplexity. Your results might be a bit strange...


In [31]:
import plotly.express as px

fig = px.scatter(tsne_df, x='tsne-2d-one', y='tsne-2d-two',
                 color='Year',
                 hover_name='Year',
                 color_continuous_scale='YlGnBu_r',
                 opacity=0.7,
                 title='Visualizzazione t-SNE')

fig.show()


Il grafico che hai generato sembra essere una visualizzazione t-SNE dei tuoi dati, colorata in base alla colonna "Year". Ecco come potresti interpretare questa visualizzazione:

1. **Distribuzione dei punti**:
   - I dati si distribuiscono in uno spazio bidimensionale in un unico grande cluster con una certa variabilità interna.
   - Non sembrano esserci distinte separazioni o cluster ben definiti, il che potrebbe suggerire che i dati non si separano nettamente in gruppi distinti in base all'anno.

2. **Colorazione in base all'anno**:
   - Il gradiente di colore mostra una transizione dai toni più scuri ai più chiari, che rappresentano un intervallo temporale dal 1960 al 2000.
   - I colori sembrano mescolarsi abbastanza uniformemente senza formare sottogruppi distanziati, indicando che non ci sono cambiamenti o tendenze radicali nel comportamento dei dati attraverso gli anni.

3. **Implicazioni**:
   - Se i dati rappresentano misurazioni o caratteristiche che si evolvono nel tempo, l'integrazione uniforme dei colori suggerisce che le variazioni anno per anno potrebbero non essere drammatiche o che l'effetto dell'anno non è il fattore dominante nella differenziazione dei dati.
   - Potrebbe anche significare che mentre ci sono differenze sottili tra i dati dei diversi anni, queste non sono abbastanza significative da formare cluster separati nello spazio t-SNE.

4. **Considerazioni sulla visualizzazione**:
   - t-SNE è un algoritmo non lineare di riduzione della dimensionalità che può a volte esagerare le vicinanze o le distanze tra i punti. Pertanto, anche se non vedi gruppi distinti, ciò non significa necessariamente che non ci siano pattern più sottili.
   - Inoltre, t-SNE ha parametri chiave (come la perplexità e il tasso di apprendimento) che possono influenzare significativamente la visualizzazione. Sperimentare con questi parametri potrebbe rivelare strutture diverse nei dati.

In conclusione, il grafico suggerisce che, almeno nello spazio t-SNE, l'anno non sembra essere un forte discriminante per questi dati. Tuttavia, per una comprensione più approfondita, potresti esplorare ulteriormente con diversi parametri di t-SNE, o potresti esaminare i dati utilizzando altre tecniche di analisi o visualizzazione per vedere se esistono altre tendenze o pattern.